# Ejercicio - Busqueda de Alojamiento en Airbnb.

Supongamos que somos un agente de [Airbnb](http://www.airbnb.com) localizado en Lisboa, y tenemos que atender peticiones de varios clientes. Tenemos un archivo llamado `airbnb.csv` (en la carpeta data) donde tenemos información de todos los alojamientos de Airbnb en Lisboa.

In [14]:
import pandas as pd
df_airbnb = pd.read_csv("./src/airbnb.csv")

In [15]:
df_airbnb.head()

,room_id,host_id,room_type,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,price
0,6499,14455,Entire home/apt,Belém,8,5.0,2,1.0,57.0
1,17031,66015,Entire home/apt,Alvalade,0,0.0,2,1.0,46.0
2,25659,107347,Entire home/apt,Santa Maria Maior,63,5.0,3,1.0,69.0
3,29248,125768,Entire home/apt,Santa Maria Maior,225,4.5,4,1.0,58.0
4,29396,126415,Entire home/apt,Santa Maria Maior,132,5.0,4,1.0,67.0


In [16]:
df_airbnb.dtypes

room_id                   int64
host_id                   int64
room_type                object
neighborhood             object
reviews                   int64
overall_satisfaction    float64
accommodates              int64
bedrooms                float64
price                   float64
dtype: object

En concreto el dataset tiene las siguientes variables:
- room_id: el identificador de la propiedad
- host_id: el identificador del dueño de la propiedad
- room_type: tipo de propiedad (vivienda completa/(habitacion para compartir/habitación privada)
- neighborhood: el barrio de Lisboa
- reviews: El numero de opiniones
- overall_satisfaction: Puntuacion media del apartamento
- accommodates: El numero de personas que se pueden alojar en la propiedad
- bedrooms: El número de habitaciones
- price: El precio (en euros) por noche

## Usando Pandas

### Caso 1.

Alicia va a ir a Lisboa durante una semana con su marido y sus 2 hijos. Están buscando un apartamento con habitaciones separadas para los padres y los hijos. No les importa donde alojarse o el precio, simplemente quieren tener una experiencia agradable. Esto significa que solo aceptan lugares con más de 10 críticas con una puntuación mayor de 4. Cuando seleccionemos habitaciones para Alicia, tenemos que asegurarnos de ordenar las habitaciones de mejor a peor puntuación. Para aquellas habitaciones que tienen la misma puntuación, debemos mostrar antes aquellas con más críticas. Debemos darle 3 alternativas.

In [19]:
import pandas as pd
df_airbnb = pd.read_csv("./src/airbnb.csv")

# Condiciones del problema
condition = (df_airbnb['reviews'] > 10) & (df_airbnb['overall_satisfaction'] > 4)
condition_2 = (df_airbnb['accommodates'] >= 4) & (df_airbnb['bedrooms'] >= 3)

# Aplicar condiciones
df_filter = df_airbnb[condition & condition_2]

# Ordenamiento
df_order = df_filter.sort_values(['overall_satisfaction', 'reviews'], ascending=[False, False])

# Top 3
df_top3 = df_order.head(3)

# Mostrar resultados
display(df_top3)


,room_id,host_id,room_type,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,price
16,44043,192830,Entire home/apt,Santa Maria Maior,316,5.0,7,3.0,80.0
2899,4132059,20604318,Entire home/apt,Misericórdia,273,5.0,6,3.0,48.0
688,785650,4141894,Entire home/apt,Santa Maria Maior,266,5.0,5,3.0,133.0


### Caso 2

Roberto es un casero que tiene una casa en Airbnb. De vez en cuando nos llama preguntando sobre cuales son las críticas de su alojamiento. Hoy está particularmente enfadado, ya que su hermana Clara ha puesto una casa en Airbnb y Roberto quiere asegurarse de que su casa tiene más críticas que las de Clara. Tenemos que crear un dataframe con las propiedades de ambos. Las id de las casas de Roberto y Clara son 97503 y 90387  respectivamente. Finalmente guardamos este dataframe como excel llamado "roberto.xls

In [24]:
import pandas as pd
df_airbnb = pd.read_csv("./src/airbnb.csv")

try:
    import openpyxl
except ImportError:
    !pip install openpyxl
    import openpyxl

# ID de las casas de Roberto y Clara
roberto_id = 97503
clara_id = 90387

# Pedido para Roberto y Clara
df_roberto = df_airbnb[df_airbnb['room_id'] == roberto_id]
df_clara = df_airbnb[df_airbnb['room_id'] == clara_id]

# Concatenar los DataFrames de Roberto y Clara
df_combinado = pd.concat([df_roberto, df_clara])

# Guardar el DataFrame combinado como un archivo Excel
excel_filename = "roberto.xls"
df_combinado.to_excel(excel_filename, index=False, engine='openpyxl')  # Añadir engine='openpyxl

# Imprimir el DataFrame combinado
display(df_combinado)

,room_id,host_id,room_type,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,price
48,97503,515924,Private room,Santo António,39,4.0,2,1.0,26.0
41,90387,490050,Private room,Santo António,204,5.0,2,1.0,46.0


### Caso 3

Diana va a Lisboa a pasar 3 noches y quiere conocer a gente nueva. Tiene un presupuesto de 50€ para su alojamiento. Debemos buscarle las 10 propiedades más baratas, dandole preferencia a aquellas que sean habitaciones compartidas *(room_type == Shared room)*, y para aquellas viviendas compartidas debemos elegir aquellas con mejor puntuación.

In [27]:
import pandas as pd
df_airbnb = pd.read_csv("./src/airbnb.csv")

# Filtar propiedades dentro del presupuesto de Diana y en Lisboa
df_diana = df_airbnb[(df_airbnb['price'] <= 50) & (df_airbnb['neighborhood'] == 'Lisboa')]

# Filtrar habitaciones compartidas y ordenar por precio y puntuación
df_diana_shared = df_diana[df_diana['room_type'] == 'Shared room'].sort_values(['price', 'overall_satisfaction'], ascending=[True, False])

# Filtrar otras habitaciones y ordenar por precio y puntuación
df_diana_other = df_diana[df_diana['room_type'] != 'Shared room'].sort_values(['price', 'overall_satisfaction'], ascending=[True, False])

# Concatenar los DataFrames para obtener las 10 propiedades más baratas
df_diana_top10 = pd.concat([df_diana_shared.head(5), df_diana_other.head(5)])

# Imprimir el DataFrame con las 10 propiedades más baratas para Diana
display(df_diana_top10)


,room_id,host_id,room_type,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,price
